In [2]:
pip install pydotmap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import sys
import re
import json
import os
import cv2
import numpy as np

from requests import get
from tqdm import tqdm
from bs4 import BeautifulSoup as soup
from concurrent.futures import ThreadPoolExecutor

from pydotmap import DotMap


class PinterestImageScraper:

    def __init__(self):
        self.json_data_list = []
        self.unique_img = []

    @staticmethod
    def clear():
        if os.name == 'nt':
            _ = os.system('cls')
        else:
            _ = os.system('clear')

    # ---------------------------------------- GET GOOGLE RESULTS ---------------------------------
    @staticmethod
    def get_pinterest_links(body):
        searched_urls = []
        html = soup(body, 'html.parser')
        links = html.select('#main > div > div > div > a')
        print('[+] saving results ...')
        for link in links:
            link = link.get('href')
            link = re.sub(r'/url\?q=', '', link)
            if link[0] != "/" and "pinterest" in link:
                searched_urls.append(link)

        return searched_urls

    # -------------------------- save json data from source code of given pinterest url -------------
    def get_source(self, url):
        try:
            res = get(url)
        except Exception as e:
            return
        html = soup(res.text, 'html.parser')
        # get json data from script tag having id initial-state
        json_data = html.find_all("script", attrs={"id": "__PWS_DATA__"})
        for a in json_data:
            self.json_data_list.append(a.string)

    # --------------------------- READ JSON OF PINTEREST WEBSITE ----------------------
    def save_image_url(self):
        print('[+] saving image urls ...')
        url_list = [i for i in self.json_data_list if i.strip()]
        if not len(url_list):
            return url_list
        url_list = []
        for js in self.json_data_list:
            try:
                data = DotMap(json.loads(js))
                urls = []
                for pin in data.props.initialReduxState.pins:
                    if isinstance(data.props.initialReduxState.pins[pin].images.get("orig"), list):
                        for i in data.props.initialReduxState.pins[pin].images.get("orig"):
                            urls.append(i.get("url"))
                    else:
                        urls.append(data.props.initialReduxState.pins[pin].images.get("orig").get("url"))

                for url in urls:
                    url_list.append(url)
            except Exception as e:
                continue
        
        return list(set(url_list))

    # ------------------------------ image hash calculation -------------------------
    def dhash(self, image, hashSize=8):
        resized = cv2.resize(image, (hashSize + 1, hashSize))
        diff = resized[:, 1:] > resized[:, :-1]
        return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

    # ------------------------------  save all downloaded images to folder ---------------------------
    def saving_op(self, var):
        url_list, folder_name = var
        if not os.path.exists(os.path.join(os.getcwd(), folder_name)):
                os.mkdir(os.path.join(os.getcwd(), folder_name))
        for img in tqdm(url_list):
            result = get(img, stream=True).content
            file_name = img.split("/")[-1]
            file_path = os.path.join(os.getcwd(), folder_name, file_name)
            img_arr = np.asarray(bytearray(result), dtype="uint8")
            image = cv2.imdecode(img_arr, cv2.IMREAD_COLOR)
            if not self.dhash(image) in self.unique_img:
                cv2.imwrite(file_path, image)
            self.unique_img.append(self.dhash(image))
            print("", end="\r")

    # ------------------------------  download images from image url list ----------------------------
    def download(self, url_list, keyword):
        folder_name = keyword
        num_of_workers = 10
        idx = len(url_list) // num_of_workers if len(url_list) > 9 else len(url_list)
        param = []
        for i in range(num_of_workers):
            param.append((url_list[((i*idx)):(idx*(i+1))], folder_name))
        with ThreadPoolExecutor(max_workers=num_of_workers) as executor:
            executor.map(self.saving_op, param)
        PinterestImageScraper.clear()

    # -------------------------- get user keyword and google search for that keywords ---------------------
    @staticmethod
    def start_scraping(key=None):
        try:
            key = input("Enter keyword: ") if key == None else key
            keyword = key + " pinterest"
            keyword = keyword.replace("+", "%20")
            url = f'http://www.google.co.in/search?hl=en&q={keyword}'
            print('[+] starting search ...')
            res = get(url)
            searched_urls = PinterestImageScraper.get_pinterest_links(res.content)
        except Exception as e:
            return []

        return searched_urls, key.replace(" ", "_")


    def make_ready(self, key=None):
        extracted_urls, keyword = PinterestImageScraper.start_scraping(key)

        print('[+] saving json data ...')
        for i in extracted_urls:
            self.get_source(i)

        # get all urls of images and save in a list
        url_list = self.save_image_url()

        # download images from saved images url
        print(f"[+] Total {len(url_list)} files available to download.")
        print()

        if len(url_list):
            try:
                self.download(url_list, keyword)
            except KeyboardInterrupt:
                return False
            return True
        
        return False


if __name__ == "__main__":
    p_scraper = PinterestImageScraper()
    is_downloaded = p_scraper.make_ready()

    if is_downloaded:
        print("\nDownloading completed !!")
    else:
        print("\nNothing to download !!")

Enter keyword: Sneaker head
[+] starting search ...
[+] saving results ...
[+] saving json data ...
[+] saving image urls ...
[+] Total 240 files available to download.








  0%|          | 0/24 [00:00<?, ?it/s]



  0%|          | 0/24 [00:00<?, ?it/s]


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]





  0%|          | 0/24 [00:00<?, ?it/s]






  0%|          | 0/24 [00:00<?, ?it/s]







  0%|          | 0/24 [00:00<?, ?it/s]








  4%|▍         | 1/24 [00:00<00:08,  2.86it/s]




  4%|▍         | 1/24 [00:00<00:09,  2.40it/s]







  4%|▍         | 1/24 [00:00<00:07,  2.96it/s]








  4%|▍         | 1/24 [00:00<00:07,  3.25it/s]








  4%|▍         | 1/24 [00:00<00:12,  1.85it/s]



  4%|▍         | 1/24 [00:00<00:15,  1.51it/s]


  4%|▍         | 1/24 [00:00<00:15,  1.52it/s]





  4%|▍         | 1/24 [00:00<00:16,  1.39it/s]






  8%|▊         | 2/24 [00:00<00:10,  2.03it/s]








  8%|▊         | 2/24 [00:00<00:09,  2.43it/s]



  8%|▊         | 2/24 [00:01<00:10,  2.01it/s]

  8%|▊         | 2/24 [00:01<00:12,  1.72it/s]







  8%|▊         | 2/24 [00:01<00:11,  1.84it/s]





  8%|▊         | 2/24 [00:01<00:11,  1.95it/s]






  8%|▊         | 2/24 [00:01<00:12,  1.69it/s]




...

100%|██████████| 24/24 [00:18<00:00,  1.26it/s]



 96%|█████████▌| 23/24 [00:19<00:00,  1.56it/s]







 83%|████████▎ | 20/24 [00:19<00:01,  2.19it/s]









 88%|████████▊ | 21/24 [00:19<00:01,  2.50it/s]


100%|██████████| 24/24 [00:19<00:00,  1.23it/s]










 92%|█████████▏| 22/24 [00:19<00:00,  2.49it/s]









 96%|█████████▌| 23/24 [00:20<00:00,  2.51it/s]









100%|██████████| 24/24 [00:20<00:00,  1.17it/s]

In [7]:
!zip -r /content/sneakers.zip /content/Sneaker_head


  adding: content/Sneaker_head/ (stored 0%)
  adding: content/Sneaker_head/5bf74229198cdd6e079ad43ace7560d5.jpg (deflated 5%)
  adding: content/Sneaker_head/dffaa16d15dbb27c5a9d99657c309c2a.jpg (deflated 5%)
  adding: content/Sneaker_head/9e43103529b97778f9c871b1af2ed3ef.jpg (deflated 7%)
  adding: content/Sneaker_head/772000aedf9e16e9d8b920382b351d1a.jpg (deflated 4%)
  adding: content/Sneaker_head/585f3defbaf9c5b91982fd5e43b27f2d.jpg (deflated 3%)
  adding: content/Sneaker_head/12bd2ba9e6aa1cc7c37c5f21ac2385d1.jpg (deflated 5%)
  adding: content/Sneaker_head/75424e9c4822838ac8fce8dc41bd6d9a.jpg (deflated 6%)
  adding: content/Sneaker_head/b42aec34639f6bd0cfc37da65c395b89.jpg (deflated 4%)
  adding: content/Sneaker_head/207757c9f8d5db08eaca6d43872a1cad.jpg (deflated 7%)
  adding: content/Sneaker_head/c79969ba3d64430c65957e85088ecd1a.jpg (deflated 5%)
  adding: content/Sneaker_head/297310407fcd35268ea4d3d3f99cc016.jpg (deflated 4%)
  adding: content/Sneaker_head/0e1a215915632f5449e0d06

In [8]:
from google.colab import files
files.download("/content/sneakers.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>